# PetFinder.my - Pawpularity Contest
## 避難所のペットの写真の人気を予測する
This is a Japanese version. English version is here. https://www.kaggle.com/chumajin/petfinder-eda-lgbm-for-starter-english-version

--------------------------------------------
このコンペは、サブタイトルのとおり、ペットの写真の人気(魅力)**Pawpularity**を予測するコンペです。

このことにより、ペットのプロフィール、写真の品質を自動的に向上させ、

野良犬や野良猫は「熱狂的な」家をはるかに早く見つけることができるようになるらしいです!

--------------------------------------------

少しでもお役に立てば、upvoteいただけたら嬉しいです。

いつもupvoteしていただける方ありがとうございます！


In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import cv2

# 1 何を予測するか ? (sample_submission.csvから見ます)

In [ ]:
sample = pd.read_csv("../input/petfinder-pawpularity-score/sample_submission.csv")
sample

**Idの写真から、Pawpularityを予測。**

Pawpularityとは、ペットの人気(魅力)を指標化したもの。

Pawpularityスコアは、他のwebやmobile、その他手法での様々なページから統計量を正規化するアルゴリズムを使って、

それぞれのペットのプロフィールページの統計から計算されます。

とのことです。

# 2. どのデータから予測するか ? (test.csvを見ます)

In [ ]:
test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")
test

  
* Id:画像ファイル名,
* Subject Focus:ペットは整頓された背景に対して際立っており、近すぎたり遠すぎたりすることはありません。,
* Eyes:両方の目が正面または正面近くを向いており、少なくとも1つの目/瞳孔が適切にクリアされています。,
* Face:正面または正面近くを向いた、きちんとクリアな顔。,
* Near:写真のかなりの部分を占める1匹のペット（写真の幅または高さの約50％以上）。,
* Action:アクション（ジャンプなど）の途中でペットを飼う。,
* Accessory:付属の物理的またはデジタルの付属品/支柱（おもちゃ、デジタルステッカーなど）、首輪と鎖を除く。,
* Group:写真に写っているペットが1匹以上。,
* Collage:デジタルレタッチされた写真（つまり、デジタルフォトフレーム、複数の写真の組み合わせ）。,
* Human:写真の中の人間。,
* Occlusion:ペットの一部をブロックする特定の望ましくないオブジェクト（つまり、人間、ケージ、または柵）。すべてのブロッキングオブジェクトがオクルージョンと見なされるわけではないことに注意してください。,
* Info:カスタム追加されたテキストまたはラベル（つまり、ペットの名前、説明）。,
* Blur:特にペットの目や顔の焦点がはっきりしていないか、ノイズが多い。ぼかしエントリの場合、「目」列は常に0に設定されます。,
* Pawpularity:ペットの人気(魅力)度


**少し注意が必要です.これらのラベルは、Pawpularityスコアの導出には使用されていません。以下詳細。**


* オプションの写真メタデータが含まれており、主要な視覚的品質と構成パラメータについて各写真に手動でラベルを付けています。
* これらのラベルは、Pawpularityスコアの導出には使用されませんが、コンテンツをよりよく理解し、写真の魅力と相互に関連付けるのに役立つ場合があります。私たちの最終目標は、インテリジェントな推奨事項（つまり、正面のペットの顔を近づける、アクセサリーを追加する、被写体の焦点を上げるなど）と写真の自動強調（つまり、明るさ、コントラスト）を生成できるAIソリューションを展開することです。より簡単に解釈できる予測。
* 必要に応じてこれらのラベルを使用し、オプションで中間/補足モデルを作成して、写真からラベルを予測することができます。補足モデルが適切な場合は、AIツールにも統合する場合があります。
* 私たちの制作システムでは、動的にスコアリングされる新しい写真には写真ラベルが含まれません。Pawpularity予測モデルで写真ラベルのスコアが必要な場合は、中間モデルを使用してそのようなパラメーターを導出してから、最終モデルにフィードします。

**そのため、Pawpularity Scoreは、それぞれのペットのプロフィールページ(おそらく写真込み)から計算されていることに注意が必要です。**

# 3. train.csvの中身を見てみる

In [ ]:
train = pd.read_csv("../input/petfinder-pawpularity-score/train.csv")
train

In [ ]:
train_path = "../input/petfinder-pawpularity-score/train"

## 一番上のIDを見てみる

In [ ]:
path = os.path.join(train_path,train["Id"].iloc[0]+".jpg")

In [ ]:
img = cv2.imread(path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)

かわいい。。。Pawpularity　= 63



# 3.1 日本語説明を表示してみる

In [ ]:
train.iloc[0]

In [ ]:
explain_dict ={    
"Id":"画像ファイル名",
"Subject Focus":"ペットは整頓された背景に対して際立っており、近すぎたり遠すぎたりすることはありません。",
"Eyes":"両方の目が正面または正面近くを向いており、少なくとも1つの目/瞳孔が適切にクリアされています。",
"Face":"正面または正面近くを向いた、きちんとクリアな顔。",
"Near":"写真のかなりの部分を占める1匹のペット（写真の幅または高さの約50％以上）。",
"Action":"アクション（ジャンプなど）の途中でペットを飼う。",
"Accessory":"付属の物理的またはデジタルの付属品/支柱（おもちゃ、デジタルステッカーなど）、首輪と鎖を除く。",
"Group":"写真に写っているペットが1匹以上。",
"Collage":"デジタルレタッチされた写真（つまり、デジタルフォトフレーム、複数の写真の組み合わせ）。",
"Human":"写真の中の人間。",
"Occlusion":"ペットの一部をブロックする特定の望ましくないオブジェクト（つまり、人間、ケージ、または柵）。すべてのブロッキングオブジェクトがオクルージョンと見なされるわけではないことに注意してください。",
"Info":"カスタム追加されたテキストまたはラベル（つまり、ペットの名前、説明）。",
"Blur":"特にペットの目や顔の焦点がはっきりしていないか、ノイズが多い。ぼかしエントリの場合、「目」列は常に0に設定されます。",
    "Pawpularity":"ペットの人気(魅力)度"
}

In [ ]:
train_jap = train.copy()

In [ ]:
train_jap.columns = train.columns.map(explain_dict)

In [ ]:
train_jap.head(3)

一番上のid

In [ ]:
tmpdf = train_jap[train_jap.index==0].T

In [ ]:
tmpdf

In [ ]:
img = cv2.imread(path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)

## train.csvで1の項目

In [ ]:
for a in tmpdf[tmpdf[0]==True].index:
    print("・" + a)

## train.csvで0の項目

In [ ]:
for a in tmpdf[tmpdf[0]==False].index:
    print("・" + a)

### 以上をまとめて関数化する ###

ここまでを1つのセルで表して、id 0を引数に変える

In [ ]:
path = os.path.join(train_path,train["Id"].iloc[0]+".jpg")

plt.figure()

img = cv2.imread(path)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)

plt.show()

tmpdf = train_jap[train_jap.index==0].T

print("--------ペットのPawpularity------------")

print(tmpdf[0].iloc[-1])

print("--------1の項目------------")

for a in tmpdf[tmpdf[0]==True].index:
    print("・ " + a)


print("")



print("--------0の項目------------")

for a in tmpdf[tmpdf[0]==False].index:
    print("・ " + a)



In [ ]:
def showimg(id):
    
    plt.figure()
    path = os.path.join(train_path,train["Id"].iloc[id]+".jpg")

    img = cv2.imread(path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    
    plt.show()

    tmpdf = train_jap[train_jap.index==id].T
    
    print("--------ペットのPawpularity------------")

    print(tmpdf[id].iloc[-1])

    print("--------1の項目------------")

    for a in tmpdf[tmpdf[id]==True].index:
        
        if a == "Pawpularity":
            continue
        
        print("・ " + a)


    print("")



    print("--------0の項目------------")

    for a in tmpdf[tmpdf[id]==False].index:
        
        if a == "Pawpularity":
            continue
        print("・ " + a)



In [ ]:
showimg(1)

In [ ]:
showimg(2)

## Pawpularity100を見てみる

In [ ]:
train[train["Pawpularity"]==100]

In [ ]:
showimg(19)

In [ ]:
showimg(50)

2匹映っていても100
決まった項目が満たされているからといって、100点を出すわけではなさそう

## Pawpularity0はないので、1を見てみる

In [ ]:
train[train["Pawpularity"]==1]

In [ ]:
showimg(2442)

In [ ]:
showimg(3232)

In [ ]:
showimg(4235)

## Pawpularity10ごとに１枚見ていきます。

In [ ]:
tmpdf3 = train.groupby("Pawpularity").head(1).sort_values("Pawpularity").reset_index()
tmpdf3

In [ ]:
tmpdf4 = tmpdf3.iloc[::10,:]
tmpdf4

In [ ]:
for a in tmpdf4["index"]:
    showimg(a)
    print("")
    print("#################################################")

# 4. LGBMでお試しのsubmit

In [ ]:
train

In [ ]:
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

# 4.1 Kfold

In [ ]:
folds = train.copy()
Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds["Pawpularity"])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.groupby(['fold', "Pawpularity"]).size())

# 4.2 Main

In [ ]:
import lightgbm as lgb

In [ ]:
import random

def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

SEED = 42
fix_seed(SEED)

In [ ]:
features = train.columns.to_list()[1:-1]
features

In [ ]:
target = "Pawpularity"

In [ ]:
# example of parameters
lgbm_params = {
    'objective': 'rmse', # Binary classification : 2値分類ではこれを使う
    'seed': 42, # random seed : これを固定すると、再現性が出る
    'metric': 'rmse', 
    'learning_rate': 0.01,
    'max_bin': 800, # depth
    'num_leaves': 80, # leaves,
    "verbose":-1
}

In [ ]:
from sklearn.metrics import mean_squared_error

def rmsescore(y_true,y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
test

In [ ]:
scores = []
allpreds = []

allvaliddf = pd.DataFrame()


for fold in range(5):
    
    fix_seed(SEED) # for repetability

    p_train = folds[folds["fold"] != fold]
    p_val = folds[folds["fold"] == fold]

    p_train = p_train.reset_index(drop=True)
    p_val = p_val.reset_index(drop=True)

    lgb_train = lgb.Dataset(p_train[features], p_train[target])
    lgb_eval = lgb.Dataset(p_val[features], p_val[target])



    model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval,
                      verbose_eval=50,  # Learning result output every 50 iterations : 50イテレーション毎に学習結果出力
                      num_boost_round=1000,  # Specify the maximum number of iterations : 最大イテレーション回数指定
                      early_stopping_rounds=100, # Early stopping number : early stoppingを採用するiteration回数
                     
                      
                     )

    import pickle

    model_name = f"LGBMmodel{fold}.bin"

    # saving model
    pickle.dump(model, open(model_name, 'wb'))

    # loading model
    model = pickle.load(open(model_name, 'rb'))

    # predicting validation value
    oof_pred = model.predict(p_val[features])


    scores.append(rmsescore(p_val[target],oof_pred))

    # predicting for test_X
    preds = model.predict(test[features])

    #preds2 = np.where(preds>=0.5,1,0)
    
    allpreds.append(preds)
    
    # out of fold : oof
    p_val["preds"] = oof_pred
    
    allvaliddf = pd.concat([allvaliddf,p_val])

In [ ]:
scores

#### mean score

In [ ]:
np.mean(scores)

#### out of fold score

In [ ]:
rmsescore(allvaliddf[target],allvaliddf["preds"])

## 4.3 for submit

In [ ]:
allpreds = np.mean(allpreds,axis=0)

In [ ]:
sample["Pawpularity"] = allpreds

In [ ]:
sample.to_csv("submission.csv",index=False)

In [ ]:
sample

ここまで見ていただいてありがとうございます。
**upvote**いただけたら嬉しいです！